# Eat Safe, Love

## Notebook Set Up

In [34]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [36]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [38]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [40]:
# review the collections in our database
collections = db.list_collection_names()

# Print the collections
print("Collections in the 'uk_food' database:", collections)

Collections in the 'uk_food' database: ['establishments']


In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [10]:
# Find the establishments with a hygiene score of 20
query = establishments.find({"HygieneScore": 20})

# Use count_documents to display the number of documents in the result
count = establishments.count_documents({"HygieneScore": 20})
print(f"Number of establishments with a hygiene score of 20: {count}")

# Check if there are any results
if count > 0:
    # Display the first document in the results using pprint
    first_document = query.limit(1).next()
    print("First document with hygiene score of 20:")
    pprint(first_document)
else:
    print("No establishments found with a hygiene score of 20.")

Number of establishments with a hygiene score of 20: 0
No establishments found with a hygiene score of 20.


In [11]:
# Find the establishments with a hygiene score of 20
query = establishments.find({"HygieneScore": 20})

# Convert the result to a list
hygiene_score_20_list = list(query)

# Convert the result to a Pandas DataFrame
df_hygiene_score_20 = pd.DataFrame(hygiene_score_20_list)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df_hygiene_score_20.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df_hygiene_score_20.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [13]:
# Find the establishments with London as the Local Authority and RatingValue >= 4
query = establishments.find({
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
})

# Use count_documents to display the number of documents in the result
count = establishments.count_documents({
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
})
print(f"Number of establishments in London with a RatingValue >= 4: {count}")

# Check if any results exist
if count > 0:
    # Display the first document in the results using pprint
    first_document = query.limit(1).next()  # This will fetch the first document
    print("First establishment with a RatingValue >= 4 in London:")
    pprint(first_document)
else:
    print("No establishments found in London with a RatingValue >= 4.")

Number of establishments in London with a RatingValue >= 4: 0
No establishments found in London with a RatingValue >= 4.


In [14]:
# Find the establishments with London as the Local Authority and RatingValue >= 4
query = establishments.find({
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
})

# Convert the query result to a list
london_rating_4_list = list(query)

# Convert the result to a Pandas DataFrame
df_london_rating_4 = pd.DataFrame(london_rating_4_list)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df_london_rating_4.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df_london_rating_4.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [16]:
# Coordinates of "Penang Flavours" (new restaurant)
latitude = 51.083812 
longitude = 1.195625

# Define the degree range for the search
degree_search = 0.01

# Query to find establishments within the given range, with RatingValue = 5
query = {
    "RatingValue": 5,
    "Latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "Longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}

# Sort by HygieneScore (ascending) and limit to 5 results
sort = [("HygieneScore", 1)]  # 1 for ascending order
limit = 5

# Execute the query and sort the results
top_5_establishments = establishments.find(query).sort(sort).limit(limit)

# Print the results
print("Top 5 establishments with a RatingValue of 5, sorted by lowest HygieneScore:")
for establishment in top_5_establishments:
    pprint(establishment)



Top 5 establishments with a RatingValue of 5, sorted by lowest HygieneScore:


In [17]:
# Convert the query result to a list
top_5_list = list(query)

# Convert result to Pandas DataFrame
df_top_5_establishments = pd.DataFrame(top_5_list)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df_top_5_establishments.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df_top_5_establishments.head(10))

Number of rows in the DataFrame: 3
First 10 rows of the DataFrame:
             0
0  RatingValue
1     Latitude
2    Longitude


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [19]:
# Create the aggregation pipeline
pipeline = [
    # 1. Match establishments with a hygiene score of 0
    {
        "$match": {
            "HygieneScore": 0
        }
    },
    # 2. Group by LocalAuthorityName and count the number of establishments in each area
    {
        "$group": {
            "_id": "$LocalAuthorityName",  # Group by LocalAuthorityName
            "count": {"$sum": 1}  # Count the number of establishments
        }
    },
    # 3. Sort by count in descending order (highest to lowest)
    {
        "$sort": {"count": -1}
    }
]

# Execute the aggregation pipeline
result = establishments.aggregate(pipeline)

# Convert the result to a list
result_list = list(result)

# Print the number of documents in the result
print(f"Number of Local Authorities with hygiene score of 0: {len(result_list)}")

# Convert result to Pandas DataFrame for easier analysis
df = pd.DataFrame(result_list)

# Display the number of rows in the DataFrame
print(f"\nNumber of rows in the DataFrame: {df.shape[0]}")

# Display the DataFrame (automatically shown as a table in Jupyter)
df.head(10)


Number of Local Authorities with hygiene score of 0: 0

Number of rows in the DataFrame: 0


""


In [20]:
# Convert the result list to a Pandas DataFrame
df_local_authorities = pd.DataFrame(result_list)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df_local_authorities.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df_local_authorities.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []
